In [19]:
class Data:
    def __init__(self, data_dir="data/FB15k-237/", reverse=False):
        self.train_data = self.load_data(data_dir, "train", reverse=reverse)
        self.valid_data = self.load_data(data_dir, "valid", reverse=reverse)
        self.test_data = self.load_data(data_dir, "test", reverse=reverse)
        self.data = self.train_data + self.valid_data + self.test_data
        self.entities = self.get_entities(self.data)
        self.relations = self.get_relations(self.data)
        self.train_relations = self.get_relations(self.train_data)
        self.valid_relations = self.get_relations(self.valid_data)
        self.test_relations = self.get_relations(self.test_data)

    @staticmethod
    def load_data(data_dir, data_type="train", reverse=False):
        with open("%s%s.txt" % (data_dir, data_type), "r") as f:
            data = f.read().strip().split("\n")
            data = [i.split() for i in data]
            if reverse:
                data += [[i[2], i[1] + "_reverse", i[0]] for i in data]
        return data

    @staticmethod
    def get_relations(data):
        relations = sorted(list(set([d[1] for d in data])))
        return relations

    @staticmethod
    def get_entities(data):
        entities = sorted(list(set([d[0] for d in data] + [d[2] for d in data])))
        return entities


In [21]:
! pip3 install wandb

In [23]:
import wandb

wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [41]:
wandb.init(
    # set the wandb project where this run will be logged
    project="v3",
)

Hits @1,▁▂▂▃▃▃▄▄▄▅▅▅▆▆▇▇██
Hits @10,▁▂▃▄▅▅▅▆▆▆▇▇▇▇████
Hits @3,▁▂▃▄▄▄▅▅▆▆▆▆▇▇▇███
Mean reciprocal rank,▁▂▃▄▄▄▅▅▅▆▆▆▆▇▇███
T norm,▁▂▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████
train_mean_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train_step_loss,█▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Hits @1,0.09438
Hits @10,0.25012
Hits @3,0.15279


In [36]:
! pip3 install tucker_riemopt

In [37]:
import torch
import numpy as np
from torch import nn
from torch.nn.init import xavier_normal_
from tucker_riemopt import SFTucker

from torch.optim import Optimizer
from tucker_riemopt import SFTuckerRiemannian

In [42]:
class SFTuckER:
    def __init__(self, d, d1, d2):
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.rank = (d2, d1, d1)
        self.E = torch.rand((len(d.entities), d1), device=device)
        self.R = torch.rand((len(d.relations), d2), device=device)
        self.W = torch.tensor(np.random.uniform(-1, 1, (d2, d1, d1)), dtype=torch.float, device=device)
        
    def parameters(self):
        return nn.ParameterList([self.W, self.E, self.R])

    def init(self):
        xavier_normal_(self.E.data)
        xavier_normal_(self.R.data)
        # with torch.no_grad():
        #     self.E.weight.data = torch.linalg.qr(self.E.weight)[0]
        #     self.R.weight.data = torch.linalg.qr(self.R.weight)[0]

    def forward(self, e_idx, r_idx):
        relations = self.R[r_idx, :]
        subjects = self.E[e_idx, :]
        preds = torch.einsum("abc,da->dbc", self.W, relations)
        preds = torch.bmm(subjects.view(-1, 1, subjects.shape[1]), preds).view(-1, subjects.shape[1])
        preds = preds @ self.E.T
        return torch.sigmoid(preds)


class RGD(Optimizer):
    def __init__(self, model_parameters, rank, max_lr):
        self.rank = rank
        self.max_lr = max_lr
        self.lr = max_lr
        self.direction = None
        self.loss = None

        defaults = dict(rank=rank, max_lr=self.max_lr, lr=self.lr)
        params = model_parameters
        super().__init__(params, defaults)

    def fit(self, loss_fn, model, normalize_grad=False):
        x_k = SFTucker(model.W.data, [model.R.data], num_shared_factors=2, shared_factor=model.E.data)
        rgrad, self.loss = SFTuckerRiemannian.grad(loss_fn, x_k)
        rgrad_norm = rgrad.norm().detach()

        if normalize_grad:
            normalizer = normalize_grad / rgrad_norm
        else:
            normalizer = 1

        self.direction = normalizer * rgrad
        return rgrad_norm

    @torch.no_grad()
    def step(self):
        W, E, R = self.param_groups[0]["params"]

        x_k = self.direction.point
        x_k = (-self.param_groups[0]["lr"]) * self.direction + SFTuckerRiemannian.TangentVector(x_k)
        x_k = x_k.construct().round(self.rank)

        W.data.add_(x_k.core - W)
        R.data.add_(x_k.regular_factors[0] - R)
        E.data.add_(x_k.shared_factor - E)

In [43]:
import numpy as np
import torch
import time
from collections import defaultdict
from torch.optim.lr_scheduler import ExponentialLR
import argparse


def get_loss_fn(e_idx, r_idx, targets, criterion, C = 0):
    def loss_fn(T: SFTucker):
        relations = T.regular_factors[0][r_idx, :]
        subjects = T.shared_factor[e_idx, :]
        preds = torch.einsum("abc,da->dbc", T.core, relations)
        preds = torch.bmm(subjects.view(-1, 1, subjects.shape[1]), preds).view(-1, subjects.shape[1])
        preds = preds @ T.shared_factor.T
        wandb.log({'T norm': T.norm()})
        return criterion(torch.sigmoid(preds), targets) + C * T.norm()

    return loss_fn


class Experiment:
    def __init__(self, learning_rate=0.0005, ent_vec_dim=200, rel_vec_dim=200,
                 num_iterations=500, batch_size=1024, decay_rate=0., label_smoothing=0.):
        self.learning_rate = learning_rate
        self.ent_vec_dim = ent_vec_dim
        self.rel_vec_dim = rel_vec_dim
        self.num_iterations = num_iterations
        self.batch_size = batch_size
        self.decay_rate = decay_rate
        self.label_smoothing = label_smoothing
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.criterion = torch.nn.BCELoss()

    def get_data_idxs(self, data):
        data_idxs = [(self.entity_idxs[data[i][0]], self.relation_idxs[data[i][1]], self.entity_idxs[data[i][2]]) for i
                     in range(len(data))]
        return data_idxs

    def get_er_vocab(self, data):
        er_vocab = defaultdict(list)
        for triple in data:
            er_vocab[(triple[0], triple[1])].append(triple[2])
        return er_vocab

    def get_batch(self, er_vocab, er_vocab_pairs, idx):
        batch = er_vocab_pairs[idx:idx + self.batch_size]
        targets = np.zeros((len(batch), len(d.entities)))
        for idx, pair in enumerate(batch):
            targets[idx, er_vocab[pair]] = 1.
        targets = torch.FloatTensor(targets).to(self.device)
        return np.array(batch), targets

    def evaluate(self, model, data):
        hits = []
        ranks = []
        for i in range(10):
            hits.append([])

        test_data_idxs = self.get_data_idxs(data)
        er_vocab = self.get_er_vocab(self.get_data_idxs(d.data))

        losses = []
        np.random.shuffle(test_data_idxs)
        for i in range(0, len(test_data_idxs), self.batch_size):
            data_batch, targets = self.get_batch(er_vocab, test_data_idxs, i)
            e1_idx = torch.tensor(data_batch[:, 0]).to(self.device)
            r_idx = torch.tensor(data_batch[:, 1]).to(self.device)
            e2_idx = torch.tensor(data_batch[:, 2]).to(self.device)

            targets = ((1.0 - self.label_smoothing) * targets) + (1.0 / targets.size(1))

            predictions = model.forward(e1_idx, r_idx)

            losses.append(self.criterion(predictions, targets).item())

            for j in range(data_batch.shape[0]):
                filt = er_vocab[(data_batch[j][0], data_batch[j][1])]
                target_value = predictions[j, e2_idx[j]].item()
                predictions[j, filt] = 0.0
                predictions[j, e2_idx[j]] = target_value

            sort_values, sort_idxs = torch.sort(predictions, dim=1, descending=True)

            sort_idxs = sort_idxs.cpu().numpy()
            for j in range(data_batch.shape[0]):
                rank = np.where(sort_idxs[j] == e2_idx[j].item())[0][0]
                ranks.append(rank + 1)

                for hits_level in range(10):
                    if rank <= hits_level:
                        hits[hits_level].append(1.0)
                    else:
                        hits[hits_level].append(0.0)

        wandb.log({'val_loss': np.mean(losses)})
        wandb.log({'Hits @10': np.mean(hits[9])})
        wandb.log({'Hits @3': np.mean(hits[2])})
        wandb.log({'Hits @1': np.mean(hits[0])})
        wandb.log({'Mean reciprocal rank': np.mean(1. / np.array(ranks))})
#         print('val_loss:', np.mean(losses))
#         print('Hits @10: {0}'.format(np.mean(hits[9])))
#         print('Hits @3: {0}'.format(np.mean(hits[2])))
#         print('Hits @1: {0}'.format(np.mean(hits[0])))
#         print('Mean reciprocal rank: {0}'.format(np.mean(1. / np.array(ranks))))

    def train_and_eval(self):
        print("Training the TuckER model...")
        self.entity_idxs = {d.entities[i]: i for i in range(len(d.entities))}
        self.relation_idxs = {d.relations[i]: i for i in range(len(d.relations))}

        train_data_idxs = self.get_data_idxs(d.train_data)
        print("Number of training data points: %d" % len(train_data_idxs))

        model = SFTuckER(d, self.ent_vec_dim, self.rel_vec_dim)

        model.init()

        opt = RGD(model.parameters(), (self.rel_vec_dim, self.ent_vec_dim, self.ent_vec_dim), self.learning_rate)
        if self.decay_rate:
            scheduler = ExponentialLR(opt, self.decay_rate)

        er_vocab = self.get_er_vocab(train_data_idxs)
        er_vocab_pairs = list(er_vocab.keys())

        print("Starting training...")
        for it in range(1, self.num_iterations + 1):
            print('Epoch:', it)
            start_train = time.time()
            losses = []
            np.random.shuffle(er_vocab_pairs)
            for j in range(0, len(er_vocab_pairs), self.batch_size):
                data_batch, targets = self.get_batch(er_vocab, er_vocab_pairs, j)
                opt.zero_grad()
                e1_idx = torch.tensor(data_batch[:, 0]).to(self.device)
                r_idx = torch.tensor(data_batch[:, 1]).to(self.device)

                targets = ((1.0 - self.label_smoothing) * targets) + (1.0 / targets.size(1))

                loss_fn = get_loss_fn(e1_idx, r_idx, targets, self.criterion)
                grad_norm = opt.fit(loss_fn, model)
                opt.step()
                opt.zero_grad(set_to_none=True)

                loss = opt.loss.detach()
                wandb.log({'train_step_loss': loss.item()})
#                 print(j / self.batch_size, loss)

                losses.append(loss.item())
            if self.decay_rate:
                scheduler.step()
#             print('time:', time.time() - start_train)
            wandb.log({'train_mean_loss': np.mean(losses)})
#             print(np.mean(losses))
            with torch.no_grad():
#                 print("Validation:")
                self.evaluate(model, d.valid_data)
#                 if it % 5 == 0:
#                     print("Test:")
#                     self.evaluate(model, d.test_data)

In [ ]:
dataset = "FB15k-237"
num_iterations = 500
batch_size = 2048
lr = 1e9
dr = 1.0
edim = 200
rdim = 200
label_smoothing = 0.1

data_dir = "/kaggle/input/tucker-data/TuckER_data/%s/" % dataset
torch.backends.cudnn.deterministic = True
seed = 20
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
d = Data(data_dir=data_dir, reverse=True)
experiment = Experiment(num_iterations=num_iterations, batch_size=batch_size, learning_rate=lr,
                        decay_rate=dr, ent_vec_dim=edim, rel_vec_dim=rdim, label_smoothing=label_smoothing)
experiment.train_and_eval()

Training the TuckER model...
Number of training data points: 544230
Starting training...
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
Epoch: 10
Epoch: 11
Epoch: 12
Epoch: 13
Epoch: 14
Epoch: 15
Epoch: 16
Epoch: 17
Epoch: 18
Epoch: 19
Epoch: 20
Epoch: 21
Epoch: 22
Epoch: 23
Epoch: 24
Epoch: 25
Epoch: 26
Epoch: 27
Epoch: 28
Epoch: 29
Epoch: 30
Epoch: 31
Epoch: 32
Epoch: 33
Epoch: 34
Epoch: 35
Epoch: 36
Epoch: 37
Epoch: 38
Epoch: 39
Epoch: 40
Epoch: 41
Epoch: 42
Epoch: 43
Epoch: 44
Epoch: 45
Epoch: 46
Epoch: 47
Epoch: 48
Epoch: 49
Epoch: 50
Epoch: 51
Epoch: 52
Epoch: 53
Epoch: 54
Epoch: 55
Epoch: 56
Epoch: 57
Epoch: 58
Epoch: 59
Epoch: 60
Epoch: 61
Epoch: 62
Epoch: 63
Epoch: 64
Epoch: 65
Epoch: 66
Epoch: 67
Epoch: 68
Epoch: 69
Epoch: 70
Epoch: 71
Epoch: 72
Epoch: 73
Epoch: 74
Epoch: 75
Epoch: 76
Epoch: 77
Epoch: 78
Epoch: 79
Epoch: 80
Epoch: 81
Epoch: 82
Epoch: 83
Epoch: 84
Epoch: 85
Epoch: 86
Epoch: 87
Epoch: 88
Epoch: 89
Epoch: 90
Epoch: 91
Epoch: 92
